In [51]:
from utils import *
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
import pickle
from sklearn.metrics import mean_squared_error

In [31]:
# Try how it works in 1D first

In [32]:
# 2D using location


In [53]:
points = get_points('../data/points_train.csv')
points['lat'] = points.geometry.y
points['lon'] = points.geometry.x

X = points[['lat', 'lon']].values
y = points['wait'].values

In [34]:
rbf = RBF([1.0, 1.0], length_scale_bounds=(1e-05, 100000.0)) # using anisotripic kernel
gp = GaussianProcessRegressor(kernel=rbf, n_restarts_optimizer=1, random_state=42)

In [35]:
gp.fit(X, y)

/home/till/miniconda3/envs/heatchmap/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/till/miniconda3/envs/heatchmap/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:445: ConvergenceWarning: The optimal value found for dimension 1 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


GaussianProcessRegressor(kernel=RBF(length_scale=[1, 1]),
                         n_restarts_optimizer=10, random_state=42)

In [40]:
with open('models/gp_model.pkl', 'wb') as f:
    pickle.dump(gp, f)
    

In [54]:
points['pred'], points['std'] = gp.predict(X, return_std=True)
mean_squared_error(points['wait'], points['pred'])

2416.085795086453

In [55]:
# validation
val = get_points('../data/points_val.csv')
val['lat'] = val.geometry.y
val['lon'] = val.geometry.x

X = val[['lat', 'lon']].values

In [56]:
val['pred'], val['std'] = gp.predict(X, return_std=True)
mean_squared_error(val['wait'], val['pred'])

2813.2594491587747

In [50]:
val

,lat,lon,wait,geometry,pred,std
0,7.054400e+06,8.023770e+05,15.0,POINT (802376.965 7054399.896),15.0,0.00001
1,5.912645e+06,1.651197e+06,45.0,POINT (1651197.223 5912644.839),45.0,0.00001
2,6.776958e+06,2.180546e+04,20.0,POINT (21805.461 6776958.131),20.0,0.00001
3,6.845958e+06,7.575608e+05,15.0,POINT (757560.806 6845957.574),0.0,1.00000
4,5.847815e+06,5.417158e+05,20.0,POINT (541715.815 5847814.594),0.0,1.00000
...,...,...,...,...,...,...
3290,7.235660e+06,1.079760e+06,5.0,POINT (1079760.270 7235659.622),5.0,0.00001
3291,6.329757e+06,1.229131e+06,18.0,POINT (1229130.942 6329757.095),0.0,1.00000
3292,6.289130e+06,2.146618e+06,17.0,POINT (2146618.298 6289130.346),0.0,1.00000
3293,7.462261e+06,1.050819e+06,10.0,POINT (1050819.002 7462260.775),10.0,0.00001


In [36]:
# 3D using location and time when the spot was recorded (might use single ratings of spots here instead averaged)
